In [1]:
import os
import pygame
import random

# 一般設定
DIMENSION = 16 # 地圖的行(列)數
TILESIZE = 20 # 方格邊長
#################### TODO 1: 調整地雷數 #########################
MINE_NUM = 80
################################################################
RESOLUTION = (DIMENSION * TILESIZE, DIMENSION * TILESIZE) # 地圖的長寬
TITLE = 'PySweep' # 遊戲名稱
FPS = 60 # 遊戲每秒的幀數

# 顏色設定
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
DARKGREY = (40, 40, 40)
LIGHTGREY = (100, 100, 100)
GREEN = (0, 255, 0)
DARKGREEN = (0, 200, 0)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
YELLOW = (255, 255, 0)

# 遊戲標誌
icon_path = "mine.png"
if os.path.exists(icon_path):
    icon = pygame.image.load(icon_path)
    pygame.display.set_icon(icon)

pygame 2.6.0 (SDL 2.28.4, Python 3.11.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
class Tile(pygame.sprite.Sprite):
    """格子內部的渲染"""
    mine_image = None
    def __init__(self, x, y, size):
        super().__init__()
        self.image = pygame.Surface((size, size))
        self.inside = pygame.Surface((size - 2, size - 2))
        self.rect = self.image.get_rect()
        self.rect.topleft = (x, y)

        # 載入炸彈的圖片
        if Tile.mine_image is None:
            try:
                # 載入地雷圖片並縮放到適當大小
                original_image = pygame.image.load("mine.png")
                Tile.mine_image = pygame.transform.scale(original_image, (size - 4, size - 4))
            except:
                Tile.mine_image = pygame.Surface((size - 4, size - 4))
                Tile.mine_image.fill(RED)

        self.image.fill(BLACK)
        self.inside.fill(LIGHTGREY)
        self.image.blit(self.inside, (2, 2))

        self.is_mine = False
        self.is_revealed = False
        self.is_flagged = False
        self.neighboring_mines = 0

    # 格子裡的顏色與數字
    def reveal(self):
        if not self.is_flagged:
            self.is_revealed = True
            self.inside.fill(WHITE)
            if not self.is_mine and self.neighboring_mines > 0:
                font = pygame.font.SysFont("calibri", 14)
                text = font.render(str(self.neighboring_mines), True, BLACK)
                self.inside.blit(text, (self.rect.width / 4, self.rect.height / 4))
            self.image.blit(self.inside, (2, 2))

    def draw_mine(self):
        """畫出炸彈"""
        self.inside.fill(WHITE)
        self.inside.blit(Tile.mine_image, (2, 2))
        self.image.blit(self.inside, (2, 2))


    def draw_explosion(self, frame):
        """繪製爆炸效果"""
        self.inside.fill(WHITE)

        if frame == 1:
            pygame.draw.circle(self.inside, YELLOW, (TILESIZE // 2 - 1, TILESIZE // 2- 1), TILESIZE // 4)
        elif frame == 2:
            pygame.draw.circle(self.inside, RED, (TILESIZE // 2 - 1, TILESIZE // 2 - 1), TILESIZE // 3)
        elif frame == 3:
            pygame.draw.circle(self.inside, YELLOW, (TILESIZE // 2 - 1, TILESIZE // 2 - 1), TILESIZE // 2 - 2)
        elif frame == 4:
            points = [
                (TILESIZE//2-1, 2), (TILESIZE//2-1, TILESIZE-4),
                (2, TILESIZE//2-1), (TILESIZE-4, TILESIZE//2-1),
            ]
            pygame.draw.lines(self.inside, RED, False, points, 2)
        elif frame == 5:
            self.inside.fill(RED)

        self.image.blit(self.inside, (2, 2))

    # 旗子
    def toggle_flag(self):
        if not self.is_revealed:
            self.is_flagged = not self.is_flagged
            if self.is_flagged:
                self.inside.fill(YELLOW)
            else:
                self.inside.fill(LIGHTGREY)
            self.image.blit(self.inside, (2, 2))
            return True if self.is_flagged else False
        return None


class Grid(pygame.sprite.Group):
    """格子內部的處理"""
    def __init__(self, rows, cols, tile_size):
        super().__init__()
        self.rows = rows
        self.cols = cols
        self.tile_size = tile_size
        self.tiles = []

        # 爆炸動畫控制
        self.explosion_frame = 0
        self.explosion_time = 0
        self.is_exploding = False
        self.animation_speed = 100
        self.delay_before_explosion = 500
        self.explosion_started = False

        for row in range(rows):
            tile_row = []
            for col in range(cols):
                tile = Tile(col * tile_size, row * tile_size, tile_size)
                tile_row.append(tile)
                self.add(tile)
            self.tiles.append(tile_row)

        self.place_mines()

    def place_mines(self, num_mines = MINE_NUM):
        """放置地雷"""
        mines_placed = 0  # 已放置的地雷數量
        while mines_placed < num_mines:  # 直到放滿指定的地雷數量 不然就一直放
###################### TODO 3: 使得每次地雷都會隨機產生 ##########
            row = mines_placed // (DIMENSION // 2)
            col = (mines_placed % (DIMENSION // 2)) * 2 + (row % 2)
################################################################
            tile = self.tiles[row][col]
            if not tile.is_mine:  # 確保該格子尚未是地雷
                tile.is_mine = True  # 就可以設為地雷
                mines_placed += 1
        self.calculate_neighbors()  # 計算每個格子周圍的地雷數量

    def calculate_neighbors(self):
        """數鄰近格子有幾顆炸彈"""
        directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
        for row in range(self.rows):
            for col in range(self.cols):
                tile = self.tiles[row][col]
                if tile.is_mine:
                    continue
                mine_count = 0
                for dr, dc in directions:
                    r, c = row + dr, col + dc
                    if 0 <= r < self.rows and 0 <= c < self.cols and self.tiles[r][c].is_mine:
                        mine_count += 1
                tile.neighboring_mines = mine_count

    def reveal_tile(self, row, col):
        """揭開玩家選擇的格子"""
        tile = self.tiles[row][col]
        if tile.is_flagged:  # 如果該格子已被標記，就不能按開
            return False
        if not tile.is_revealed:  # 揭開該格子
            tile.reveal()
            if tile.is_mine:  # 如果是地雷
                return True  # 遊戲結束
            elif tile.neighboring_mines == 0:  # 如果該格子周圍沒有地雷
                self.reveal_neighbors(row, col)  # 自動揭開附近的格子
        return False

    def reveal_all_mines(self):
        """顯示所有地雷"""
        for row in self.tiles:
            for tile in row:
                if tile.is_mine:
                    tile.draw_mine()
        self.is_exploding = True
        self.explosion_time = pygame.time.get_ticks()

    def update(self, current_time):
        """更新爆炸動畫"""
        if self.is_exploding and not self.explosion_started:
            if current_time - self.explosion_time >= self.delay_before_explosion:
                self.explosion_started = True
                self.explosion_time = current_time
                self.explosion_frame = 0

        elif self.explosion_started:
            if current_time - self.explosion_time >= self.animation_speed:
                self.explosion_frame += 1
                self.explosion_time = current_time

                if self.explosion_frame <= 5:
                    # 同時讓所有地雷的爆炸
                    for row in self.tiles:
                        for tile in row:
                            if tile.is_mine:
                                tile.draw_explosion(self.explosion_frame)

    def reveal_neighbors(self, row, col):
        """直接揭開一片沒有炸彈的區域"""
        directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
        for dr, dc in directions:
            r, c = row + dr, col + dc
            if 0 <= r < self.rows and 0 <= c < self.cols:
                neighbor_tile = self.tiles[r][c]
                if not neighbor_tile.is_revealed and not neighbor_tile.is_flagged:
                    neighbor_tile.reveal()
                    if neighbor_tile.neighboring_mines == 0:
                        self.reveal_neighbors(r, c)

    def check_win(self):
        """
        看看每一個格子裡面：
            如果不是地雷的格子還沒被揭開
            那就還沒獲勝
        """
        for row in self.tiles:
            for tile in row:
                if not tile.is_mine and not tile.is_revealed:
                    return False
        return True

In [5]:
class Game:
    """遊戲本體類別"""
    def __init__(self): # 遊戲初始化
        pygame.init()
        self.screen = pygame.display.set_mode(RESOLUTION)
        pygame.display.set_caption(TITLE)
        self.clock = pygame.time.Clock()
        self.grid = Grid(DIMENSION, DIMENSION, TILESIZE)

        # 遊戲狀態
        self.game_started = False
        self.game_over = False
        self.won = False
        self.start_time = 0
        self.flags_count = 0

        # 字體
        self.font = pygame.font.SysFont("calibri", 14)
        self.big_font = pygame.font.SysFont("calibri", 40)

        # 底下資訊欄
        self.info_surface = pygame.Surface((RESOLUTION[0], 30))

    def new(self): # 產生新的一局遊戲
        self.grid = Grid(DIMENSION, DIMENSION, TILESIZE)
        self.game_over = False  # Reset game-over state
        self.game_started = False
        self.won = False
        self.flags_count = 0

    def run(self): # 執行遊戲迴圈
        while True:
            self.clock.tick(FPS)
            current_time = pygame.time.get_ticks()
            if self.event():
                return
            self.grid.update(current_time)
            self.draw(current_time)

    def draw(self, current_time): # 渲染
        self.screen.fill(DARKGREY)
        self.grid.draw(self.screen)

        # 讓下面的資訊欄會改變
        self.info_surface.fill(WHITE)
        if self.game_started and not self.game_over:
            game_time = int((current_time - self.start_time) / 1000)
            time_text = self.font.render("Time: " + str(game_time), True, BLACK)
            flags_text = self.font.render("Flags: " + str(self.flags_count), True, BLACK)
            self.info_surface.blit(time_text, (0, 0))
            self.info_surface.blit(flags_text, (RESOLUTION[0] // 2, 0))

        # 展示出勝負的結果
        if self.game_over:
            if self.won:
                end_text = self.big_font.render("You win!", True, BLUE)
            else:
                end_text = self.big_font.render("You lose!", True, RED)
            text_pos = ((RESOLUTION[0] - end_text.get_width()) // 2,
                       (RESOLUTION[1] - 30 - end_text.get_height()) // 2)
            self.screen.blit(end_text, text_pos)
        self.screen.blit(self.info_surface, (0, DIMENSION * TILESIZE))
        pygame.display.flip()

    def event(self): # 事件偵測
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return True  # 遊戲停止
            elif event.type == pygame.MOUSEBUTTONDOWN and not self.game_over:
                x, y = event.pos
                row = y // TILESIZE
                col = x // TILESIZE
                if event.button == 1:  # 左鍵
                    if not self.game_started:
                        self.game_started = True
                        self.start_time = pygame.time.get_ticks() # 開始計時
                    mine_hit = self.grid.reveal_tile(row, col)
                    if mine_hit:
#################### TODO 2: 踩到地雷遊戲停止 ####################
################################################################
                        self.grid.reveal_all_mines()
                    elif self.grid.check_win():
                        self.game_over = True
                        self.won = True
                elif event.button == 3:  # 右鍵
#################### TODO 4: 按下右鍵標記正確方格 ################
                    self.grid.tiles[col][row].toggle_flag()
################################################################
        return False
game = Game()

while True:
    game.new()
    game.run()
    pygame.quit()  # 停止 pygame
    break  # 跳出遊戲迴圈